# Linear Systems Stability

## Objective

Study numerical stability of linear system solvers:
- Gaussian elimination with/without pivoting
- Normal equations vs QR for least squares
- Matrix inversion vs direct solve
- Ill-conditioned systems (Hilbert matrix)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

from utils.linear_algebra_utils import (
    gaussian_elimination, gaussian_elimination_with_pivoting,
    solve_normal_equations, solve_qr, matrix_inverse_solve,
    create_hilbert_matrix, create_ill_conditioned_matrix
)
from utils.error_metrics import forward_error, backward_error, condition_number

np.random.seed(42)
plt.rcParams["figure.figsize"] = (12, 6)

## Experiment 1: Gaussian Elimination Without Pivoting

Can fail catastrophically even on well-conditioned systems.

In [ ]:
# Example where no pivoting fails
A = np.array([[1e-20, 1.0], [1.0, 1.0]])
b = np.array([1.0, 2.0])

print("Matrix A:")
print(A)
print(f"\nCondition number: {np.linalg.cond(A):.2e}")

try:
    x_no_pivot = gaussian_elimination(A.copy(), b.copy(), pivot=False)
    print(f"\nNo pivoting: x = {x_no_pivot}")
    print(f"Residual: {np.linalg.norm(A @ x_no_pivot - b):.2e}")
except Exception as e:
    print(f"\nNo pivoting failed: {e}")

x_pivot = gaussian_elimination_with_pivoting(A.copy(), b.copy())
print(f"\nWith pivoting: x = {x_pivot}")
print(f"Residual: {np.linalg.norm(A @ x_pivot - b):.2e}")

x_exact = np.linalg.solve(A, b)
print(f"\nNumPy solve: x = {x_exact}")

## Experiment 2: Hilbert Matrix (Ill-Conditioned)

In [ ]:
# Study error growth with condition number
sizes = np.arange(2, 13)
cond_numbers = []
errors_ge = []
errors_numpy = []

for n in sizes:
    H = create_hilbert_matrix(n)
    x_true = np.ones(n)
    b = H @ x_true
    
    cond = np.linalg.cond(H)
    cond_numbers.append(cond)
    
    x_ge = gaussian_elimination_with_pivoting(H.copy(), b.copy())
    x_np = np.linalg.solve(H, b)
    
    err_ge = np.linalg.norm(x_ge - x_true) / np.linalg.norm(x_true)
    err_np = np.linalg.norm(x_np - x_true) / np.linalg.norm(x_true)
    
    errors_ge.append(err_ge)
    errors_numpy.append(err_np)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.semilogy(sizes, cond_numbers, "o-", linewidth=2)
plt.xlabel("Matrix size")
plt.ylabel("Condition number")
plt.title("Hilbert Matrix Conditioning")
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.loglog(cond_numbers, errors_ge, "o-", label="Gaussian elim", linewidth=2)
plt.loglog(cond_numbers, errors_numpy, "s-", label="NumPy", linewidth=2)
eps = np.finfo(np.float64).eps
plt.loglog(cond_numbers, np.array(cond_numbers) * eps, "k--", label=r"$\kappa \varepsilon$", alpha=0.5)
plt.xlabel("Condition number")
plt.ylabel("Relative error")
plt.title("Error vs Conditioning")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../plots/04_hilbert_conditioning.png", dpi=150, bbox_inches="tight")
plt.show()

## Experiment 3: Least Squares - Normal Equations vs QR

In [ ]:
# Compare normal equations vs QR
m, n = 100, 20
condition_numbers_test = np.logspace(1, 10, 15)
errors_normal = []
errors_qr = []

for kappa in condition_numbers_test:
    A = create_ill_conditioned_matrix(n, kappa)
    A_tall = np.vstack([A, A])  # Make it overdetermined
    x_true = np.random.randn(n)
    b = A_tall @ x_true + 1e-10 * np.random.randn(2*n)
    
    try:
        x_normal = solve_normal_equations(A_tall, b)
        err_normal = np.linalg.norm(x_normal - x_true) / np.linalg.norm(x_true)
    except:
        err_normal = np.nan
    
    try:
        x_qr = solve_qr(A_tall, b)
        err_qr = np.linalg.norm(x_qr - x_true) / np.linalg.norm(x_true)
    except:
        err_qr = np.nan
    
    errors_normal.append(err_normal)
    errors_qr.append(err_qr)

plt.figure(figsize=(10, 6))
plt.loglog(condition_numbers_test, errors_normal, "o-", label="Normal equations", linewidth=2)
plt.loglog(condition_numbers_test, errors_qr, "s-", label="QR factorization", linewidth=2)
plt.loglog(condition_numbers_test, condition_numbers_test * eps, "k--", label=r"$\kappa \varepsilon$", alpha=0.5)
plt.xlabel("Condition number of A")
plt.ylabel("Relative error")
plt.title("Least Squares: Normal Equations vs QR")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig("../plots/04_least_squares_comparison.png", dpi=150, bbox_inches="tight")
plt.show()

print("Normal equations square the condition number!")
print(f"For κ(A) = 1e6, κ(A^T A) ≈ {1e6**2:.2e}")

## Experiment 4: Matrix Inversion vs Direct Solve

In [ ]:
# Compare explicit inversion vs direct solve
n = 50
A = np.random.randn(n, n)
b = np.random.randn(n)

import time

# Direct solve
t0 = time.time()
x_solve = np.linalg.solve(A, b)
t_solve = time.time() - t0

# Explicit inversion
t0 = time.time()
x_inv = matrix_inverse_solve(A, b)
t_inv = time.time() - t0

print(f"Direct solve time: {t_solve*1000:.2f} ms")
print(f"Inversion time: {t_inv*1000:.2f} ms")
print(f"\nError (direct solve): {np.linalg.norm(A @ x_solve - b):.2e}")
print(f"Error (inversion): {np.linalg.norm(A @ x_inv - b):.2e}")
print(f"\nConclusion: Never explicitly invert matrices!")

## Key Takeaways

1. **Always use pivoting** in Gaussian elimination
2. **Condition number** determines achievable accuracy: error ≈ κ(A)ε
3. **Normal equations** square the condition number - avoid for ill-conditioned problems
4. **QR factorization** preserves condition number - preferred for least squares
5. **Never explicitly invert** matrices - slower and less accurate